In [10]:
import pandas as pd
from utils.analysis import main as load_and_analyze_homepass
import os
import geopandas as gpd
import numpy as np



In [6]:
folder_path = r"C:\Users\dicky\OneDrive\Dokumen\Magang\Latihan Streamlit\website analisis\data"
geojson_files = [f for f in os.listdir(folder_path) if f.endswith('.geojson')]
data = []
df = pd.DataFrame(data)
df
# kelurahan_list = df['kelurahan'].sort_values().unique()


""


In [24]:
folder_path = r"C:\Users\dicky\OneDrive\Dokumen\Magang\Latihan Streamlit\website analisis\data"

# Fungsi untuk load dan hitung analisis TAM SAM SOM
def load_and_analyze_homepass(total_odp=329, odp_capacity=16):
    folder_path = r"C:\Users\dicky\OneDrive\Dokumen\Magang\Latihan Streamlit\website analisis\data"
    geojson_files = [f for f in os.listdir(folder_path) if f.endswith('.geojson')]
    data = []

    for file in geojson_files:
        gdf = gpd.read_file(os.path.join(folder_path, file))
        kelurahan = file.replace("Homepass Kelurahan ", "").replace(".geojson", "").lower().strip()
        homepass = len(gdf)
        data.append({'kelurahan': kelurahan, 'homepass': homepass})

    df = pd.DataFrame(data)

    # Hitung TAM, SAM, SOM
    total_homepass = df['homepass'].sum()
    df['odp_float'] = df['homepass'] / total_homepass * total_odp
    df['odp_floor'] = np.floor(df['odp_float']).astype(int)
    df['sisa'] = df['odp_float'] - df['odp_floor']

    sisa_odp = total_odp - df['odp_floor'].sum()
    df = df.sort_values(by='sisa', ascending=False)
    df.iloc[:sisa_odp, df.columns.get_loc('odp_floor')] += 1

    df['SAM'] = df['odp_floor'] * odp_capacity
    df['SOM'] = df['SAM'] * 0.3

    df['SAM'] = df['SAM'].astype(int)
    df['SOM'] = df['SOM'].round(0).astype(int)
    df = df.rename(columns={'odp_floor': 'ODP'})

    df = df.sort_values(by='SOM', ascending=False).reset_index(drop=True)
    df['ranking'] = df['SOM'].rank(ascending=False).astype(int)

    rate = df['SOM'].mean()
    df['kategori_potensi'] = df['SOM'].apply(
        lambda x: '🔥 High Potential' if x >= rate else '❄️ Low Potential'
    )

    return df[['ranking', 'kelurahan', 'homepass', 'ODP', 'SAM', 'SOM', 'kategori_potensi']]

df = load_and_analyze_homepass()

In [27]:
df

,ranking,kelurahan,homepass,ODP,SAM,SOM,kategori_potensi
0,1,mojolangu,7618,54,864,259,🔥 High Potential
1,2,jatimulyo,6640,47,752,226,🔥 High Potential
2,3,merjosari,5783,41,656,197,🔥 High Potential
3,4,lowokwaru,5351,38,608,182,🔥 High Potential
4,5,tunjungsekar,5034,36,576,173,🔥 High Potential
5,6,tulusrejo,4120,29,464,139,🔥 High Potential
6,7,tlogomas,3607,25,400,120,❄️ Low Potential
7,8,tunggulwulung,2483,18,288,86,❄️ Low Potential
8,9,dinoyo,2416,17,272,82,❄️ Low Potential
9,10,tasik madu,1694,12,192,58,❄️ Low Potential


In [25]:
print(df.columns)

Index(['ranking', 'kelurahan', 'homepass', 'ODP', 'SAM', 'SOM',
       'kategori_potensi'],
      dtype='object')


In [26]:
kelurahan_list = df['kelurahan'].sort_values().unique()
kelurahan_list

array(['dinoyo', 'jatimulyo', 'ketawanggede', 'lowokwaru', 'merjosari',
       'mojolangu', 'sumbersari', 'tasik madu', 'tlogomas', 'tulusrejo',
       'tunggulwulung', 'tunjungsekar'], dtype=object)